<a href="https://colab.research.google.com/github/rjlallana/MAFAtoYOLO/blob/main/YOLO_mafa_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el dataset

In [ ]:
# google colab gpu
import torch
print(torch.cuda.get_device_name(torch.cuda.current_device()))
!rm -r /content/sample_data

## Descargar el script para pasar las anotaciones del formato propio que usa el dataset MAFA al formato que usa YOLO

In [ ]:
!git clone https://rjlallana@github.com/rjlallana/MAFAtoYOLO.git
%cd /content/MAFAtoYOLO/

fatal: destination path 'MAFAtoYOLO' already exists and is not an empty directory.
/content/MAFAtoYOLO


## Descargar el dataset MAFA

In [ ]:
%cd /content/MAFAtoYOLO
!gdown --id 1uN0a4P0wAFwJLid_r7VHFs0KUcizIRGN # train labels
!gdown --id 1Fu1C1O8ok-Z7r8XSWoTb9yB_2_5w6BDt # test labels
!gdown --id 17bRIiaGyrKLEDQOV2RlqbPQ9TyCZxq9k # train images
!gdown --id 1jJHdmmscqxvNQ2dxKUrLaHqW3w1Yo_9S # test images

/content/MAFAtoYOLO
Downloading...
From: https://drive.google.com/uc?id=1uN0a4P0wAFwJLid_r7VHFs0KUcizIRGN
To: /content/MAFAtoYOLO/MAFA-Label-Test.zip
100% 182k/182k [00:00<00:00, 5.69MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fu1C1O8ok-Z7r8XSWoTb9yB_2_5w6BDt
To: /content/MAFAtoYOLO/MAFA-Label-Train.zip
100% 947k/947k [00:00<00:00, 15.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=17bRIiaGyrKLEDQOV2RlqbPQ9TyCZxq9k
To: /content/MAFAtoYOLO/train-images.zip
1.96GB [00:20, 94.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jJHdmmscqxvNQ2dxKUrLaHqW3w1Yo_9S
To: /content/MAFAtoYOLO/test-images.zip
466MB [00:06, 69.9MB/s]


## Descomprimir los archivos y borrar los .zip

In [ ]:
!unzip -qq MAFA-Label-Test.zip
!unzip -qq MAFA-Label-Train.zip
!unzip -qq test-images.zip -d test
!unzip -qq train-images.zip -d train
!rm *.zip
!rm *.txt

## Crear el archivo de configuaracion del Dataset
Este archivo YAML describe la estructura de directorios del dataset.
* Cada ruta en train, val, test lleva a una carpeta con un fichero de texto con las rutas las imagenes que usa.
* El numero de clases (nc)
* Y una lista de los nombres de las clases


In [ ]:
yaml = """\
train: ../MAFAtoYOLO/train/
val: ../MAFAtoYOLO/val/
test: ../MAFAtoYOLO/test/

nc: 3
names: ['No mask', 'Mask', 'Mask incorrect']
"""
with open('/content/MAFAtoYOLO/mafa.yaml', 'w') as writefile:
    writefile.write(yaml)

!cat /content/MAFAtoYOLO/mafa.yaml

train: ../MAFAtoYOLO/train/
val: ../MAFAtoYOLO/val/
test: ../MAFAtoYOLO/test/

nc: 3
names: ['No mask', 'Mask', 'Mask incorrect']


## Ejecucion del script 
Pasar las anotaciones del formato propio que usa el dataset MAFA al formato que usa YOLO

In [1]:
!ls

LabelTestAll.mat   MAFA.zip	  YOLO_mafa_dataset_colab.ipynb  images   test
LabelTrainAll.mat  MAFAtoYOLO.7z  YOLO_mafa_dataset_wsl.ipynb	 main.py  train


In [2]:
!python main.py

Traceback (most recent call last):
  File "main.py", line 582, in <module>
    main()
  File "main.py", line 475, in main
    create_yolo_structure()
  File "main.py", line 421, in create_yolo_structure
    os.mkdir('images')
FileExistsError: [Errno 17] File exists: 'images'


# Descargar el modelo YOLO

In [ ]:
%cd /content
!git clone https://github.com/rjlallana/yolo
!pip install -qr yolo/requirements.txt

/content
Cloning into 'yolo'...
remote: Enumerating objects: 401, done.
remote: Total 401 (delta 0), reused 0 (delta 0), pack-reused 401
Receiving objects: 100% (401/401), 2.96 MiB | 31.94 MiB/s, done.
Resolving deltas: 100% (186/186), done.
     |████████████████████████████████| 636 kB 27.5 MB/s 
     |████████████████████████████████| 776.7 MB 4.4 kB/s 
     |████████████████████████████████| 12.7 MB 87 kB/s 
     |████████████████████████████████| 1.5 MB 40.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.


Por defecto Google Colab tiene una version más reciente de TensorFlow que la que usa el modelo de YOLO implementado, y a la hora de entrenar el modelo en esta versión falla.
Por lo tanto descargamos una version anterior, downgrade: ``pytorch 1.8.0+cu101 -> 1.7.1+cu101``


In [ ]:
# Downgrade pytorch 1.8.0+cu101 -> 1.7.1+cu101
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

import torch 
print(torch.__version__)

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 14 kB/s 
     |████████████████████████████████| 12.8 MB 71 kB/s 
     |████████████████████████████████| 7.6 MB 20.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.1
    Uninstalling torchvision-0.8.1:
      Successfully uninstalled torchvision-0.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.1+cu101 which is incompatible.


1.9.0+cu102


## Descargamos los pesos de la red entrenados en el dataset COCO para hacer transferencia de aprendizaje

In [ ]:
%cd /content/yolo/weights
!bash get_pretrain.sh

/content/yolo/weights
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1871      0 --:--:-- --:--:-- --:--:--  1871
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  202M    0  202M    0     0  41.2M      0 --:--:--  0:00:04 --:--:-- 62.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2009      0 --:--:-- --:--:-- --:--:--  2009
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  142M    0  142M    0     0  60.1M      0 --:--:--  0:00:02 --:--:--  173M


# Entrenar el modelo para nuestro dataset

Usaremos la herramienta online [wandb](https://wandb.ai/site) para recoger todos las metricas de nuestro modelo y poder visualizarlas mas tarde. 

In [ ]:
# W&B API key: d0c1c5b12edb5a4b4ccceca771661a478bfd9efd
%cd /content/yolo
!pip --q install wandb # -qr requirements.txt  # install dependencies
!wandb login

/content/yolo
     |████████████████████████████████| 1.6 MB 11.1 MB/s 
     |████████████████████████████████| 133 kB 48.1 MB/s 
     |████████████████████████████████| 170 kB 51.3 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Entranamiento de nuestro Modelo para el dataset MAFA, a continuacion explicacmos cada parametro
* ### --img: Tamaño de la imagen de entrada
* ### --batch: Numero de batches en los que se divira nuestro dataset, en cada epoch se elige un batch diferente (Dataset = 1024, batch = 16 --> batches de 64 imagenes, cada epoch se procesan 64 imagenes ) [blog](https://towardsdatascience.com/how-to-break-gpu-memory-boundaries-even-with-large-batch-sizes-7a9c27a400ce)
* ### --epochs: Numero total de veces que el modelo cambia sus pesos
* ### --data: Archivo con la estructura del dataset
* ### --cfg: Arquitectura del modelo
* ### --weights: Pesos iniciales que usa el modelo
* ### --hyp: Hyperparametros del modelo
* ### --name: Nombre del proyecto

In [ ]:
%cd /content/yolo
!python train.py --img 128 --batch 16 --epochs 20 --data '/content/MAFAtoYOLO/mafa.yaml' --weights './weights/yolov4_csp.pt' --hyp 'data/hyp.scratch.640.yaml' --cfg '/cfg/yolor_csp.cfg' --name yolov4_csp-mafa-old

/content/yolo
Using torch 1.7.1+cu101 CUDA:0 (Tesla K80, 11441MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./cfg/yolor_csp.cfg', data='/content/MAFAtoYOLO/mafa.yaml', device='', epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.640.yaml', image_weights=False, img_size=[128, 128], local_rank=-1, log_imgs=16, multi_scale=False, name='yolov4_csp-mafa-old', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/yolov4_csp-mafa-old', single_cls=False, sync_bn=False, total_batch_size=16, weights='./weights/yolov4_csp.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-08-14 11:16:32.873329: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_e

# Dowload weights locally to resume the training

In [ ]:
!zip -r /content/last.zip /content/yolo/runs/train/yolov4_csp-mafa-old/weights/last.pt

  adding: content/yolo/runs/train/yolov4_csp-mafa-old/weights/last.pt (deflated 7%)


In [ ]:
from google.colab import files
files.download("/content/last.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh /content/last.zip

-rw-r--r-- 1 root root 376M Aug 14 11:38 /content/last.zip


# Evaluación del rendimiento del modelo una vez acabado el entranamiento

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
#from utils.general import plot_results  # plot results.txt as results.png
from IPython.display import Image, display
display(Image('/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/results.png'))  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/test_batch0_gt.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/train_batch0.jpg', width=900)

In [ ]:
# Save weights
from google.colab import files
files.download('/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt') 

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls ./runs/exp0_yolov4-csp-results/weights

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/ScaledYOLOv4/
!python detect.py --weights /content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt --img 640 --conf 0.4 --source ../MAFAtoYOLO/test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above
import glob
from IPython.display import Image, display

for imageName in glob.glob('./inference/output/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import files
files.download('/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt')

In [ ]:
# FIN